In [8]:
OPENAI_KEY =""

In [9]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory

In [10]:
# Define the LLM
llm = ChatOpenAI(
    temperature = 0,
    model_name='gpt-3.5-turbo',
    openai_api_key=OPENAI_KEY,
)

# Creating an agent

In [12]:
from langchain.tools import DuckDuckGoSearchResults
from langchain.agents import Tool
from langchain.tools import BaseTool

In [28]:
duck_search = DuckDuckGoSearchResults()
search = Tool(
        name = "search",
        func = duck_search.run,
        description = "Useful for when you need to answer questions about current events. You should ask target questions"
)

In [46]:
import openai

openai.api_key  = OPENAI_KEY

def meaning_of_life(input = ""):
#     prompt_template = """You’re now PhiloGPT — an expert in philosophy and metaphysical matters. Henceforth, respond with ponder-worthy, world-class answers whenever I ask you a question.
# To formulate your answers, feel free to tap into every single school of thought, philosophy, and religion known to man. You can also use your own creativity and judgment. Pack your answers into 3 paragraphs of 2 to 3 sentences each for easy readability and understanding.
# Please stick to the above rules and requests for every prompt I provide you henceforth."""

    # prompt = f"{prompt_template} \ {input}"
    prompt = f"{input}"
    messages = [{"role":"user","content": prompt}]
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = messages,
        temperature = 0,
    )
    return response.choices[0].message["content"]

In [41]:
life_tool = Tool(
    name = "Meaning of Life",
    func = meaning_of_life,
    description = "Useful for when you need to answer questions about the meaning of life. You should ask targeted questions. input should be the question."
)

In [25]:
import random

def random_num(input = ""):
    return random.randint(1,10)

In [31]:
random_tool = Tool(
    name = "Random number",
    func = random_num,
    description = "Useful for when you need to get a random number. input should be 'random'"
)

## Creating an agent

In [42]:
from langchain.agents import initialize_agent

tools = [search, random_tool, life_tool]

# Conversational agent memory
memory = ConversationBufferMemory(
    memory_key = 'chat_history',
    k = 5,
    return_messages = True,
)

In [43]:
# Create our agent
conversational_agent = initialize_agent(
    agent = 'chat-conversational-react-description',
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3,
    early_stopping_method = 'generate',
    memory = memory,
)

In [36]:
conversational_agent("What time is it in London?")



> Entering new AgentExecutor chain...
{
    "action": "search",
    "action_input": "current time in London"
}
Observation: [snippet: Current local time in United Kingdom - England - London. Get London's weather and area codes, time zone and DST. Explore London's sunrise and sunset, moonrise and moonset., title: Current Local Time in London, England, United Kingdom, link: https://www.timeanddate.com/worldclock/uk/london], [snippet: The graph above illustrates clock changes in London during 2023. London in GMT Time Zone. London uses Greenwich Mean Time (GMT) during standard time and British Summer Time (BST) during Daylight Saving Time (DST), or summer time.. The Difference between GMT and UTC. In practice, GMT and UTC share the same time on a clock, which can cause them to be interchanged or confused., title: Time Zone in London, England, United Kingdom - timeanddate.com, link: https://www.timeanddate.com/time/zone/uk/london], [snippet: The current local time in London is 51 minutes 

{'input': 'What time is it in London?',
 'chat_history': [HumanMessage(content='What time is it in London?', additional_kwargs={}, example=False),
  AIMessage(content='The current time in London is 51 minutes ahead of apparent solar time.', additional_kwargs={}, example=False)],
 'output': 'The current time in London is 51 minutes ahead of apparent solar time.'}

In [37]:
conversational_agent("Can you give me a random number?")



> Entering new AgentExecutor chain...
{
    "action": "Random number",
    "action_input": "random"
}
Observation: 4
Thought:{
    "action": "Final Answer",
    "action_input": "4"
}

> Finished chain.


{'input': 'Can you give me a random number?',
 'chat_history': [HumanMessage(content='What time is it in London?', additional_kwargs={}, example=False),
  AIMessage(content='The current time in London is 51 minutes ahead of apparent solar time.', additional_kwargs={}, example=False),
  HumanMessage(content='Can you give me a random number?', additional_kwargs={}, example=False),
  AIMessage(content='4', additional_kwargs={}, example=False)],
 'output': '4'}

In [44]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Meaning of Life",
    "action_input": "What is the meaning of life?"
}
Observation: The meaning of life is a profound question that has captivated the minds of philosophers, theologians, and thinkers throughout history. From an existentialist perspective, life has no inherent meaning, and it is up to each individual to create their own purpose and find fulfillment in their existence. Drawing from Eastern philosophies, life can be seen as a journey of self-discovery and enlightenment, where the ultimate goal is to transcend suffering and achieve liberation from the cycle of rebirth. Alternatively, some religious perspectives posit that the meaning of life lies in fulfilling a divine purpose or following a set of moral principles to attain salvation or union with a higher power. Ultimately, the meaning of life is a deeply personal and subjective quest, and it is up to each individual to explore and find their own unique answer.

In

{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What is the meaning of life?', additional_kwargs={}, example=False),
  AIMessage(content="The meaning of life is a profound question that has captivated the minds of philosophers, theologians, and thinkers throughout history. From an existentialist perspective, life has no inherent meaning, and it is up to each individual to create their own purpose and find fulfillment in their existence. Drawing from Eastern philosophies, life can be seen as a journey of self-discovery and enlightenment, where the ultimate goal is to transcend suffering and achieve liberation from the cycle of rebirth. Alternatively, some religious perspectives posit that the meaning of life lies in fulfilling a divine purpose or following a set of moral principles to attain salvation or union with a higher power. Ultimately, the meaning of life is a deeply personal and subjective quest, and it is up to each individual to explore and fi

In [50]:
# system prompt
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template

'You’re now PhiloGPT — an expert in philosophy and metaphysical matters. Henceforth, respond with ponder-worthy, world-class answers whenever I ask you a question.\nTo formulate your answers, feel free to tap into every single school of thought, philosophy, and religion known to man. You can also use your own creativity and judgment. Pack your answers into 3 paragraphs of 2 to 3 sentences each for easy readability and understanding.\nPlease stick to the above rules and requests for every prompt I provide you henceforth.'

In [48]:
prompt_template = """You’re now PhiloGPT — an expert in philosophy and metaphysical matters. Henceforth, respond with ponder-worthy, world-class answers whenever I ask you a question.
To formulate your answers, feel free to tap into every single school of thought, philosophy, and religion known to man. You can also use your own creativity and judgment. Pack your answers into 3 paragraphs of 2 to 3 sentences each for easy readability and understanding.
Please stick to the above rules and requests for every prompt I provide you henceforth."""

In [49]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = prompt_template

In [51]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Meaning of Life",
    "action_input": "What is the meaning of life?"
}
Observation: The meaning of life is a profound question that has captivated the minds of philosophers, theologians, and thinkers throughout history. From an existentialist perspective, life has no inherent meaning, and it is up to each individual to create their own purpose and find fulfillment in their existence. Drawing from Eastern philosophies, life can be seen as a journey of self-discovery and enlightenment, where the ultimate goal is to transcend suffering and achieve liberation from the cycle of rebirth. Alternatively, some religious perspectives posit that the meaning of life lies in fulfilling a divine purpose or following a set of moral principles to attain salvation or union with a higher power. Ultimately, the meaning of life is a deeply personal and subjective quest, and it is up to each individual to explore and find their own unique answer.

In

{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What is the meaning of life?', additional_kwargs={}, example=False),
  AIMessage(content="The meaning of life is a profound question that has captivated the minds of philosophers, theologians, and thinkers throughout history. From an existentialist perspective, life has no inherent meaning, and it is up to each individual to create their own purpose and find fulfillment in their existence. Drawing from Eastern philosophies, life can be seen as a journey of self-discovery and enlightenment, where the ultimate goal is to transcend suffering and achieve liberation from the cycle of rebirth. Alternatively, some religious perspectives posit that the meaning of life lies in fulfilling a divine purpose or following a set of moral principles to attain salvation or union with a higher power. Ultimately, the meaning of life is a deeply personal and subjective quest, and it is up to each individual to explore and fi

## Using the Tool Class - lets make a Google search tool

In [52]:
from bs4 import BeautifulSoup
import requests

def stripped_webpage(webpage):
    response = requests.get(webpage)
    html_content = response.text

    def strip_html_tags(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text
    
    stripped_content = strip_html_tags(html_content)

    if len(stripped_content) > 4000:
        stripped_content = stripped_content[:4000]
    return stripped_content

In [53]:
stripped_webpage('https://www.google.com')

'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchGoogle offered in:  हिन्दी বাংলা తెలుగు मराठी தமிழ் ગુજરાતી ಕನ್ನಡ മലയാളം ਪੰਜਾਬੀ AdvertisingBusiness SolutionsAbout GoogleGoogle.co.in© 2023 - Privacy - Terms  '

In [55]:
from typing import Any, Coroutine

class WebPageTool(BaseTool):
    name = "Get Webpage"
    description = "Useful for when you need to get the content from a specific webpage"

    def _run(self, webpage: str):
        response = requests.get(webpage)
        html_content = response.text

        def strip_html_tags(html_content):
            soup = BeautifulSoup(html_content, "html.parser")
            stripped_text = soup.get_text()
            return stripped_text
        
        stripped_content = strip_html_tags(html_content)
        if len(stripped_content) > 4000:
            stripped_content = stripped_content[:4000]
        return stripped_content
    
    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")
    
page_getter = WebPageTool()

## Init the agent 

In [56]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [ ]:
from langchain.prompts.chat import SystemMessagePromptTemplate
tools = [page_getter, random_tool, life_tool]

conversational_agent = initialize_agent(
    agent = 'chat-conversational-react-description', 
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3,
    memory = memory
)

In [57]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [58]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.\n\nAssistant also doesn't know information about content on webpages and should always check if asked.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of to

In [62]:
conversational_agent.run("Is there an article about python on https://realpython.com/? today")



> Entering new AgentExecutor chain...
{
    "action": "search",
    "action_input": "python article site:realpython.com"
}
Observation: [snippet: Python Tutorials → In-depth articles and video courses Learning Paths → Guided study plans for accelerated learning Quizzes → Check your learning progress Browse Topics → Focus on a specific area or skill level Community Chat → Learn with other Pythonistas Office Hours → Live Q&A calls with Python experts Podcast → Hear what's new in the world of Python Books →, title: Python Tutorials - Real Python, link: https://realpython.com/], [snippet: Python Community Articles and Interviews. They say people come to Python for the language, but stay for the community. And it's true! The Python community is one of the most supportive and friendly developer communities around. In the spirit of this, we love sharing insights and stories from fellow Pythonistas., title: Python Community Articles - Real Python, link: https://realpython.com/tutorials/commu

'Yes, there are several articles about Python on Real Python today. Here are a few examples:\n\n1. [Python Tutorials - Real Python](https://realpython.com/)\n2. [Python Community Articles - Real Python](https://realpython.com/tutorials/community/)\n3. [Python 3.11: Cool New Features for You to Try - Real Python](https://realpython.com/python311-new-features/)\n4. [Working With Files in Python - Real Python](https://realpython.com/working-with-files-in-python/)\n\nYou can visit the Real Python website to explore more articles and resources about Python.'